In [0]:
import pandas as pd
#import numpy as np
from nltk.tokenize import word_tokenize 
from nltk.stem import PorterStemmer 
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [0]:
!pip install numpy==1.16.1
import numpy as np

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
cd drive/My Drive/RecommenderSystems And TopicModelling Lab2

/content/drive/My Drive/RecommenderSystems And TopicModelling Lab2


In [0]:
path="data/"
news_path=path+"nytimes_news_articles.txt"

In [0]:
stopwords=np.load(path+'stopwords.npy')
news_df = pd.read_pickle(path+'news_df')

In [0]:
freq_map={}
ps=PorterStemmer()
for article in news_df['Article']:
    word_list=word_tokenize(article)
    article_map={}
    for word in word_list:
        word=word.replace('\n','').replace('-',' ')
        if len(word)>2:
            if word.isalpha():
                if word not in stopwords:
                    word_s=ps.stem(word)
                    if word_s not in stopwords:
                      try :
                        float(word_s.replace(',',''))
                        continue
                      except :  
                        if word_s not in article_map:
                            if word_s in freq_map:
                                freq_map[word_s]+=1
                            else:
                                freq_map[word_s]=1
                            article_map[word_s]=1    
                        


In [0]:
number_of_articles=len(news_df)


In [0]:
word_to_index={}
index_to_word={}
index=0
for key in freq_map:
  word_to_index[key]=index
  index_to_word[index]=key
  index+=1  

In [0]:
#del vector_space_table

In [0]:
#del vector_space_table
vector_space_table=np.load(path+'vector_space_table2.npy')
#print("yes1")

In [0]:
number_of_terms=len(freq_map)

In [0]:
number_of_topics=70

In [0]:
w_z=np.random.rand(number_of_topics,number_of_terms)
z_d=np.random.rand(number_of_articles,number_of_topics)

In [0]:
sum_w_z=sum(w_z)
w_z/=sum_w_z
sum_z_d=sum(z_d)
z_d/=sum(z_d)

In [0]:
#del article_dict_list
article_dict_list=[]

In [0]:

for i in range(number_of_articles):
  doc_word_map={}
  for j in range(number_of_terms):
    if(vector_space_table[i][j]>0):
      doc_word_map[j]=np.zeros(number_of_topics)
  article_dict_list.append(doc_word_map)    
      

In [0]:
word_doc_map={}
for j in range(number_of_terms):
  word_doc_map[j]=[]
  for i in range(number_of_articles):
    if(vector_space_table[i][j]>0):
      word_doc_map[j].append(i)
  

In [0]:
# p=np.zeros(shape=(number_of_articles,number_of_terms,number_of_topics))
# print("yes4")

In [0]:
def E_step():
  for i in range(0,number_of_articles):
    if(i%1000==0):
      print(i)
    #list_terms=[key for key in ]
    for j in article_dict_list[i]:


      denominator=0
      for k in range(number_of_topics):
        article_dict_list[i][j][k]=z_d[i][k]*w_z[k][j]

        denominator+=article_dict_list[i][j][k]
      if (denominator !=0):
        for k in range(number_of_topics):
          article_dict_list[i][j][k]=article_dict_list[i][j][k]/denominator
      else:
        for k in range(number_of_topics):
          article_dict_list[i][j][k]=0

In [0]:
def M_step():
  #update w_z
  for k in range(number_of_topics):
    if(k%20==0):
      print(k)

    denominator=0
    for j in range(number_of_terms):
      w_z[k][j]=0
      for i in word_doc_map[j]:
        if vector_space_table[i][j]!=0:
          w_z[k][j]+=vector_space_table[i][j]*article_dict_list[i][j][k]
        denominator+=w_z[k][j]
    if(denominator!=0):
      for j in range(number_of_terms):
        w_z[k][j]/=denominator
    else:
      for j in range(number_of_terms):
        w_z[k][j]=1.0/number_of_terms
#   den_mat=np.zeros(number_of_topics)
#   #  del w_z
#   w_z=np.zeros(shape=(number_of_topics,number_of_terms))
#   for i in range(number_of_articles):
#   #     if(i%1000==0):
#   #       print(i)
#     for k in range (number_of_topics):
#       for j in article_dict_list[i]:
#           w_z[k][j]+=vector_space_table[i][j]*article_dict_list[i][j][k]
#           den_mat[k]+=w_z[k][j]
#   for k in range(number_of_topics):
#     if den_mat[k]!=0:
#       for j in range(number_of_terms):
#         w_z[k][j]/=den_mat[k]
#     else:
#       for j in range(number_of_terms):
#         w_z[k][j]=1.0/number_of_terms
#   del den_mat      
 #update z_d

  for i in range(number_of_articles):
    if(i%2000==0):
      print(i)
    for k in range(number_of_topics):
      #print('\n')
      z_d[i][k]=0
      denominator=0
      for j in article_dict_list[i]:
        #if(vector_space_table[i][j]==0):
         # print("dafuq")
        z_d[i][k]+=vector_space_table[i][j]*article_dict_list[i][j][k]
        denominator+=vector_space_table[i][j]
#       if(i%10==0 and k%10==0):
#         print(denominator)
      if (denominator!=0):
        z_d[i][k]/=denominator
#         if(i%10==0):
          
#           print(z_d[i][k])
      else:
        #print("hello")
        z_d[i][k]=1.0/number_of_topics
  return w_z     


In [0]:
num_iters=50

In [0]:
w_z=np.load(path+'w_z.npy')
z_d=np.load(path+'z_d.npy')
article_dict_list=np.load(path+'article_dict_list.npy')

In [0]:
for i in range(num_iters):
    print("Iter: "+str(i))
    E_step()
    M_step()
    if(i%5==0):
      if(i==5):
        print(z_d[0])
        print(z_d[1])
      np.save(path+'w_z',w_z)
      np.save(path+'z_d',z_d)
      np.save(path+'article_dict_list',article_dict_list)
      

In [0]:

for k in range(number_of_topics):
  print('\n'+"Topic: "+str(k+1))
  words_in_this_topic=[]
  word_indices=w_z[k,:].argsort()
  for i in range(1,11):
    print(index_to_word[word_indices[-i]]+", ",end='')
  
  


Topic: 1
china, chines, compani, said, hong, govern, beij, kong, trade, year, 
Topic: 2
sander, voter, clinton, poll, vote, state, democrat, primari, percent, win, 
Topic: 3
said, hospit, fire, bodi, report, die, peopl, death, one, day, 
Topic: 4
countri, said, govern, refuge, european, nation, europ, franc, year, intern, 
Topic: 5
said, bank, israel, compani, isra, palestinian, custom, use, secur, offer, 
Topic: 6
friend, want, just, didn, know, day, time, get, ask, never, 
Topic: 7
said, match, open, play, tenni, djokov, player, french, tournament, year, 
Topic: 8
said, attack, islam, state, kill, group, govern, forc, militari, war, 
Topic: 9
appl, app, can, use, servic, like, compani, will, phone, new, 
Topic: 10
britain, european, union, british, vote, leav, london, referendum, said, europ, 
Topic: 11
dog, like, one, peopl, can, even, cloud, world, make, way, 
Topic: 12
film, show, movi, like, said, one, charact, star, watch, seri, 
Topic: 13
hit, run, game, yanke, said, inning, p

In [0]:
num_iters=50
for i in range(0, num_iters):
    print("Iter: "+str(i))
    E_step()
    w_z,z_d=M_step(w_z,z_d)
    if(i%5==0):
      np.save(path+'w_z',w_z)
      np.save(path+'z_d',z_d)
      np.save(path+'article_dict_list',article_dict_list)


In [0]:
w_z=np.load(path+'w_z.npy')